# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In this notebook, we'll explore a use-case for RAGAS' synthetic testset generation workflow using PDF files!

- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [7]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [8]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [9]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

## Data Preparation

We'll prepare our data by loading PDF files from the `/pdf` directory.

These PDFs are assumed to be already present in the directory.

In [10]:
import os
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

pdf_path = "pdf/"
loader = DirectoryLoader(pdf_path, loader_cls=PyPDFLoader)
docs = loader.load()

#pdf_files = [f for f in os.listdir(pdf_path) if f.lower().endswith('.pdf')]
#print(f'Processed {len(pdf_files)} PDF file(s) and loaded {len(docs)} sub-documents')

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 54, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/51 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/54 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/94 [00:00<?, ?it/s]

Property 'summary' already exists in node 'afc6e8'. Skipping!
Property 'summary' already exists in node 'ba0ce8'. Skipping!
Property 'summary' already exists in node 'f21643'. Skipping!
Property 'summary' already exists in node 'b1c961'. Skipping!
Property 'summary' already exists in node '059a26'. Skipping!
Property 'summary' already exists in node '5350ac'. Skipping!
Property 'summary' already exists in node '00863d'. Skipping!
Property 'summary' already exists in node '90fc31'. Skipping!
Property 'summary' already exists in node '55e1f1'. Skipping!
Property 'summary' already exists in node '85fdd9'. Skipping!
Property 'summary' already exists in node '413a87'. Skipping!
Property 'summary' already exists in node 'd8c741'. Skipping!
Property 'summary' already exists in node '032be7'. Skipping!
Property 'summary' already exists in node '502f9d'. Skipping!
Property 'summary' already exists in node 'a91f2e'. Skipping!
Property 'summary' already exists in node '6de51e'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/17 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/112 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'f96ed6'. Skipping!
Property 'summary_embedding' already exists in node 'ba0ce8'. Skipping!
Property 'summary_embedding' already exists in node 'afc6e8'. Skipping!
Property 'summary_embedding' already exists in node '55e1f1'. Skipping!
Property 'summary_embedding' already exists in node 'f21643'. Skipping!
Property 'summary_embedding' already exists in node '00863d'. Skipping!
Property 'summary_embedding' already exists in node '5350ac'. Skipping!
Property 'summary_embedding' already exists in node '57f2fd'. Skipping!
Property 'summary_embedding' already exists in node '90fc31'. Skipping!
Property 'summary_embedding' already exists in node 'df0e39'. Skipping!
Property 'summary_embedding' already exists in node 'd8c741'. Skipping!
Property 'summary_embedding' already exists in node 'b1c961'. Skipping!
Property 'summary_embedding' already exists in node '059a26'. Skipping!
Property 'summary_embedding' already exists in node '85fdd9'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 106, relationships: 4399)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_ind_kg.json")
ai_ind_kg = KnowledgeGraph.load("ai_ind_kg.json")
ai_ind_kg

KnowledgeGraph(nodes: 106, relationships: 4399)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_ind_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What role does an Independent Ethics Committee...,[IND means an investigational new drug applica...,An Independent Ethics Committee (IEC) is a rev...,single_hop_specifc_query_synthesizer
1,Cud yu pleese explane how the FDA's guidelins ...,"[Chemistry, manufacturing, and control informa...","The FDA's guidelines require that, as appropri...",single_hop_specifc_query_synthesizer
2,What are the general requirements for using an...,[§ 312.40 General requirements for use of an i...,An investigational new drug may be used in a c...,single_hop_specifc_query_synthesizer
3,What role does the President play in the waive...,[(ii) Ongoing use. FDA may place an ongoing ex...,A determination by the President to waive the ...,single_hop_specifc_query_synthesizer
4,Whaat is the role of the Center for Drug Evalu...,"[[52 FR 8831, Mar. 19, 1987, as amended at 55 ...",The Center for Drug Evaluation and Research is...,single_hop_specifc_query_synthesizer
5,"How do clinical procedures and the chemistry, ...",[<1-hop>\n\n(g) A description of clinical proc...,Clinical procedures are essential in the IND a...,multi_hop_abstract_query_synthesizer
6,What clinical procedures and drug substance in...,[<1-hop>\n\n(g) A description of clinical proc...,"For an investigational new drug application, a...",multi_hop_abstract_query_synthesizer
7,What are the clinical procedures and chemistry...,[<1-hop>\n\n(g) A description of clinical proc...,For an Investigational New Drug (IND) applicat...,multi_hop_abstract_query_synthesizer
8,how 21 CFR Part 312 affect responsibilities of...,"[<1-hop>\n\nChemistry, manufacturing, and cont...",21 CFR Part 312 outlines the responsibilities ...,multi_hop_specific_query_synthesizer
9,Wht r the genral requirments for an IND to b i...,[<1-hop>\n\n§ 312.40 General requirements for ...,The general requirements for an IND to be in e...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/51 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/54 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/94 [00:00<?, ?it/s]

Property 'summary' already exists in node '289086'. Skipping!
Property 'summary' already exists in node 'abdb85'. Skipping!
Property 'summary' already exists in node 'b8247a'. Skipping!
Property 'summary' already exists in node '6adb2f'. Skipping!
Property 'summary' already exists in node '1861ab'. Skipping!
Property 'summary' already exists in node '9e8354'. Skipping!
Property 'summary' already exists in node '892a27'. Skipping!
Property 'summary' already exists in node '11c438'. Skipping!
Property 'summary' already exists in node '6a05c0'. Skipping!
Property 'summary' already exists in node 'd99c0d'. Skipping!
Property 'summary' already exists in node '569be5'. Skipping!
Property 'summary' already exists in node '39f625'. Skipping!
Property 'summary' already exists in node 'ac04a8'. Skipping!
Property 'summary' already exists in node '0637ed'. Skipping!
Property 'summary' already exists in node '5ee70c'. Skipping!
Property 'summary' already exists in node '93d861'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '289086'. Skipping!
Property 'summary_embedding' already exists in node '49a1f7'. Skipping!
Property 'summary_embedding' already exists in node 'abdb85'. Skipping!
Property 'summary_embedding' already exists in node '93d861'. Skipping!
Property 'summary_embedding' already exists in node 'd99c0d'. Skipping!
Property 'summary_embedding' already exists in node 'b8247a'. Skipping!
Property 'summary_embedding' already exists in node '6adb2f'. Skipping!
Property 'summary_embedding' already exists in node '11c438'. Skipping!
Property 'summary_embedding' already exists in node '892a27'. Skipping!
Property 'summary_embedding' already exists in node '6a05c0'. Skipping!
Property 'summary_embedding' already exists in node '9e8354'. Skipping!
Property 'summary_embedding' already exists in node '1861ab'. Skipping!
Property 'summary_embedding' already exists in node 'ac04a8'. Skipping!
Property 'summary_embedding' already exists in node 'e82457'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is an institutional review board?,[IND means an investigational new drug applica...,"An institutional review board (IRB), as define...",single_hop_specifc_query_synthesizer
1,"What does the FDA require for chemistry, manuf...","[Chemistry, manufacturing, and control informa...",The FDA requires a section describing the comp...,single_hop_specifc_query_synthesizer
2,What are the FDA requirements for an IND to go...,[§ 312.40 General requirements for use of an i...,An IND goes into effect thirty days after FDA ...,single_hop_specifc_query_synthesizer
3,What 21 CFR 312.42(b) say?,[Grounds for imposition of clinical hold — 21 ...,21 CFR 312.42(b) is part of the regulations co...,single_hop_specifc_query_synthesizer
4,Where should an Investigational New Drug Appli...,[<1-hop>\n\n§ 312.140 Address for corresponden...,An Investigational New Drug Application for dr...,multi_hop_abstract_query_synthesizer
5,What are the general requirements for the use ...,[<1-hop>\n\n§ 312.40 General requirements for ...,"According to 21 CFR Part 312, the general requ...",multi_hop_abstract_query_synthesizer
6,What are the general requirements for the use ...,[<1-hop>\n\n§ 312.40 General requirements for ...,"According to 21 CFR Part 312, specifically § 3...",multi_hop_abstract_query_synthesizer
7,How do the general requirements for the use of...,[<1-hop>\n\n§ 312.40 General requirements for ...,The general requirements for the use of an inv...,multi_hop_abstract_query_synthesizer
8,What are the responsibilities of a sponsor und...,[<1-hop>\n\nIND means an investigational new d...,"Under 21 CFR Part 312, a sponsor has several r...",multi_hop_specific_query_synthesizer
9,What are the responsibilities of sponsors unde...,"[<1-hop>\n\nChemistry, manufacturing, and cont...","Under 21 CFR Part 312, sponsors have several r...",multi_hop_specific_query_synthesizer


## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "FDA Investigational New Drug Application!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="FDA Investigational New Drug Application!"
)

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="IND Application"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "Create a checklist for the IND application"})

'1. Cover sheet (Form FDA-1571)\n   - Name, address, and telephone number of the sponsor\n   - Date of the application\n   - Name of the investigational new drug\n   - Identification of the phase(s) of the clinical investigation to be conducted\n   - Commitment not to begin clinical investigations until the IND is in effect\n\n2. Signature\n   - Signature of the sponsor or authorized representative\n   - If not residing in the U.S., include name and address of an attorney or authorized official in the U.S.\n\n3. Table of contents\n\n4. Introductory statement and general investigational plan\n   - Name of the drug and active ingredients\n   - Pharmacological class and structural formula (if known)\n   - Formulation of dosage form(s) and route of administration\n   - Broad objectives and planned duration of proposed clinical investigations\n   - Brief summary of previous human experience with the drug and reference to other INDs if applicable\n\n5. Description of clinical procedures and 

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [35]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

coherence_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "coherence": {
                "Is the response coherent with the retrieved data or the query?"  
                "Is the response easy to read and logically structured and well-written?"            
            }          
        },
        "llm" : eval_llm
    }
)

## LangSmith Evaluation

In [36]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        coherence_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'passionate-selection-81' at:
https://smith.langchain.com/o/b2bbbf41-ce3b-46fa-8cc0-7de9bb768aa9/datasets/5ba0e029-51de-4103-9721-d14f6ba83f15/compare?selectedSessions=d1ad7380-6b01-4de9-ad29-43449c2e80ef




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.coherence,execution_time,example_id,id
0,How IND go into effect and what happen if inve...,An IND goes into effect thirty days after the ...,None,An IND goes into effect 30 days after the FDA ...,1,0,1,2.451554,189a374a-4f24-4d7c-bcc0-838b345ac812,dd6f79ba-d301-4e6d-acab-0233325912ae
1,What role does 21 CFR Part 312 play in the inv...,I don't know.,None,21 CFR Part 312 outlines the regulations for t...,0,0,0,2.049997,240ab4ed-ca86-4fdd-9f98-23d711198bce,743fc994-6366-45c9-b774-31c705a50fc9
2,What are the responsibilities of sponsors unde...,I don't know.,None,"Under 21 CFR Part 312, sponsors have several r...",0,0,0,2.254050,14c7e1bd-32d2-43e6-9f10-09663567c78c,4d69d45b-8db1-447e-ae67-3cf707f20b1d
3,What are the responsibilities of a sponsor und...,The responsibilities of a sponsor under 21 CFR...,None,"Under 21 CFR Part 312, a sponsor has several r...",1,0,1,4.235959,d85f3a3d-5321-4958-bb3b-a96638993c41,100a4ca8-507f-4af1-916a-7498d49deac0
4,How do the general requirements for the use of...,The general requirements for the use of an inv...,None,The general requirements for the use of an inv...,1,1,1,3.579515,5c3df2c3-0ef7-4bb4-a94a-117a1f0324bb,1d0aed2b-c6ab-41dc-8537-fdc7071c6fbe
5,What are the general requirements for the use ...,The general requirements for the use of an inv...,None,"According to 21 CFR Part 312, specifically § 3...",1,1,1,2.392881,914cf9e2-81b3-4c6f-bbe9-78d00321735d,8cf71e61-dbe3-4c91-9468-40dc31dbcbaf
6,What are the general requirements for the use ...,The general requirements for the use of an inv...,None,"According to 21 CFR Part 312, the general requ...",1,0,1,3.888910,37e5cd28-e9e6-43e6-964e-d5b87676794f,98a86389-839c-4990-9c38-a822c7096d59
7,Where should an Investigational New Drug Appli...,The Investigational New Drug Application for d...,None,An Investigational New Drug Application for dr...,1,1,1,2.252324,87e5e6f1-8878-49d5-a9b0-3d9f2b43ea93,c8db70d0-2165-4635-a3f5-03ae26ae9624
8,What 21 CFR 312.42(b) say?,I don't know.,None,21 CFR 312.42(b) is part of the regulations co...,0,0,0,1.021734,3a59096f-ed26-4002-93b8-868b8a5f34c2,0754cc1b-adec-43bc-a6c9-a14960a901da
9,What are the FDA requirements for an IND to go...,An IND goes into effect thirty days after the ...,None,An IND goes into effect thirty days after FDA ...,1,0,1,1.662763,2d5d8fea-61f4-4b4d-8ac4-8ac7230734e9,a9c00898-c082-4856-b480-7fafc27ee80d


# 🤝 Improving the RAG Chain

In [37]:
rag_documents = docs

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

In [39]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [40]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="IND Application (Augmented)"
)

In [41]:
retriever = vectorstore.as_retriever()

In [42]:
rag_chain_augmented = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [43]:
rag_chain_augmented.invoke({"question" : "Create a checklist for the IND application"})

"Based on the provided context, here is a checklist for the IND application:\n\n1. Cover sheet (Form FDA-1571):\n   - Name, address, and telephone number of the sponsor\n   - Date of the application\n   - Name of the investigational new drug\n   - Identification of the phase or phases of the clinical investigation\n   - Commitment not to begin clinical investigations until an IND covering the investigations is in effect\n   - Commitment that an Institutional Review Board (IRB) that complies with the requirements is in place\n\n2. Statement if obligations have been transferred to a contract research organization:\n   - Name and address of the contract research organization\n   - Identification of the clinical study\n   - Listing of the obligations transferred or a general statement of the transfer\n\n3. Signature:\n   - Signature of the sponsor or the sponsor's authorized representative\n   - If applicable, countersigned by an attorney, agent, or other authorized official residing in th

In [44]:
evaluate(
    rag_chain_augmented.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        coherence_evaluatorrag_chain_augmented
    ],
    metadata={"revision_id": "coherence_chain"},
)

View the evaluation results for experiment: 'excellent-coffee-3' at:
https://smith.langchain.com/o/b2bbbf41-ce3b-46fa-8cc0-7de9bb768aa9/datasets/5ba0e029-51de-4103-9721-d14f6ba83f15/compare?selectedSessions=0460e628-edba-40c2-aaef-4793a4fc69fc




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.coherence,execution_time,example_id,id
0,How IND go into effect and what happen if inve...,An Investigational New Drug Application (IND) ...,None,An IND goes into effect 30 days after the FDA ...,0,0,1,3.064751,189a374a-4f24-4d7c-bcc0-838b345ac812,e36cfb75-6d92-4b82-8bd7-852607a605c1
1,What role does 21 CFR Part 312 play in the inv...,21 CFR Part 312 outlines the regulatory framew...,None,21 CFR Part 312 outlines the regulations for t...,1,0,1,3.069906,240ab4ed-ca86-4fdd-9f98-23d711198bce,a502fd71-6ccb-45a2-b77f-5373f0f485ba
2,What are the responsibilities of sponsors unde...,I don't know.,None,"Under 21 CFR Part 312, sponsors have several r...",0,0,0,1.229251,14c7e1bd-32d2-43e6-9f10-09663567c78c,4bde5361-ba93-4525-befd-d884ac76a128
3,What are the responsibilities of a sponsor und...,"Under 21 CFR Part 312, the responsibilities of...",None,"Under 21 CFR Part 312, a sponsor has several r...",1,0,1,5.325389,d85f3a3d-5321-4958-bb3b-a96638993c41,8d83215c-c412-4d21-ad27-e5bfd258e229
4,How do the general requirements for the use of...,The general requirements for the use of an inv...,None,The general requirements for the use of an inv...,1,0,1,2.869378,5c3df2c3-0ef7-4bb4-a94a-117a1f0324bb,c3512c5d-103e-4731-84fa-280a8bc1c021
5,What are the general requirements for the use ...,The general requirements for the use of an inv...,None,"According to 21 CFR Part 312, specifically § 3...",0,0,0,3.274460,914cf9e2-81b3-4c6f-bbe9-78d00321735d,ea84e0fc-b93c-4688-8dba-2be6d3cb6071
6,What are the general requirements for the use ...,"According to 21 CFR Part 312, the general requ...",None,"According to 21 CFR Part 312, the general requ...",0,0,0,3.276336,37e5cd28-e9e6-43e6-964e-d5b87676794f,c86c699f-8a84-46a2-9543-7a7be59b2ffb
7,Where should an Investigational New Drug Appli...,The Investigational New Drug Application for d...,None,An Investigational New Drug Application for dr...,1,1,1,2.548194,87e5e6f1-8878-49d5-a9b0-3d9f2b43ea93,b53e9ce0-d812-4c53-b19d-04c0b0f71282
8,What 21 CFR 312.42(b) say?,I don't know.,None,21 CFR 312.42(b) is part of the regulations co...,0,0,0,1.042214,3a59096f-ed26-4002-93b8-868b8a5f34c2,7f1c9382-9fef-477d-becb-b6c60ca7e5c2
9,What are the FDA requirements for an IND to go...,The FDA requirements for an IND to go into eff...,None,An IND goes into effect thirty days after FDA ...,1,1,1,2.352430,2d5d8fea-61f4-4b4d-8ac4-8ac7230734e9,25645520-b912-4807-b2b7-c97f2b23c52f
